<a href="https://colab.research.google.com/github/Joselito1junior/ai_codes_gemini/blob/main/gemini_scribe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando o SDK do Google

In [2]:
!pip install -q -U google-generativeai

Importações iniciais

In [126]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

Escolhendo o modelo

In [128]:
models = []
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    models.append(m.name)
models
choosed_model = models[0]

Cadastrando algumas regras

In [129]:
#Listagem de documentos que serão buscados
LIVROS_1 = {
    "Titulo": "Livros com um autor",
    "Regra": "SOBRENOME, Nome. Título: subtítulo (se houver). Edição. Local de publicação: Editora, ano de publicação."}

LIVROS_2 = {
    "Titulo": "Livros com mais de três autores",
    "Regra": "SOBRENOME, Nome et al.. Título: subtítulo (se houver). Edição. Local de publicação: Editora, ano de publicação."}

PERIODICOS = {
    "Titulo": "Artigos em periódicos",
    "Regra": "SOBRENOME, Nome. Título do artigo. Título do periódico, volume(número): página inicial-página final, mês de publicação, ano de publicação."}

ACADEMICOS = {
    "Titulo": "Trabalhos acadêmicos",
    "Regra": "SOBRENOME, Nome. Título do trabalho: subtítulo (se houver). Nível de ensino, Instituição de ensino, Local de apresentação, ano de apresentação."}

ONLINE = {
    "Titulo": "Conteúdo da internet",
    "Regra": "SOBRENOME, Nome. Título do conteúdo online: subtítulo (se houver). Disponível em: Endereço eletrônico. Acesso em: data de acesso."}

documents = [LIVROS_1, LIVROS_2, PERIODICOS, ACADEMICOS, ONLINE]

Criando o dataframe

In [130]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Regra"]
df

,Titulo,Regra
0,Livros com um autor,"SOBRENOME, Nome. Título: subtítulo (se houver)..."
1,Livros com mais de três autores,"SOBRENOME, Nome et al.. Título: subtítulo (se ..."
2,Artigos em periódicos,"SOBRENOME, Nome. Título do artigo. Título do p..."
3,Trabalhos acadêmicos,"SOBRENOME, Nome. Título do trabalho: subtítulo..."
4,Conteúdo da internet,"SOBRENOME, Nome. Título do conteúdo online: su..."


Criando os embeddings

In [131]:
def embed_fn(title, text):
  return genai.embed_content(model=choosed_model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

Salvando os embeddings criados no dataframe

In [132]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Regra"]), axis=1)
df

,Titulo,Regra,Embeddings
0,Livros com um autor,"SOBRENOME, Nome. Título: subtítulo (se houver)...","[0.006676407, -0.03909685, -0.034612875, -0.01..."
1,Livros com mais de três autores,"SOBRENOME, Nome et al.. Título: subtítulo (se ...","[0.012224127, -0.023320403, -0.036158156, -0.0..."
2,Artigos em periódicos,"SOBRENOME, Nome. Título do artigo. Título do p...","[0.0024109993, -0.04720702, -0.040160734, 0.00..."
3,Trabalhos acadêmicos,"SOBRENOME, Nome. Título do trabalho: subtítulo...","[0.0048586535, -0.042066917, -0.008993416, 0.0..."
4,Conteúdo da internet,"SOBRENOME, Nome. Título do conteúdo online: su...","[0.0076969783, -0.028781816, -0.024221273, -0...."


Função que realiza a consulta

In [133]:
def gerar_e_buscar_consulta(consulta, base):
  embedding_da_consulta = genai.embed_content(model=choosed_model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Regra"]

Ajuste de configuração do modelo

In [134]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

Função para criação de repostas mais dinâmicas

In [135]:
def prompt_create(consulta):
  retorno_da_consulta = gerar_e_buscar_consulta(consulta, df)
  # print(f"O tipo encontrado foi: {retorno_da_consulta}")
  prompt = f"Escreva o resultado de '{retorno_da_consulta} e explique, se possível dê um exemplo para o tipo de referência dada"
  model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
  response = model_2.generate_content(prompt)
  print(response.text)

Criação de consultas para testes

In [136]:
consulta = "Qual a forma de fazer uma referência bibliográfica de um livro escrito por apenas um autor?"
# consulta = "Qual a forma de fazer uma referência bibliográfica de um livro escrito por mais de 3 autores?"
# consulta = "Qual a forma de fazer uma referência bibliográfica apartir de um conteúdo na internet?"
# consulta = "Me dê um exemplo de referência bibliográfica de um texto estraido de uma tese de mestrado"
# consulta = "Qual a forma de fazer uma referência bibliográfica referênciando um site na internet?"
# consulta = "Qual a forma o tipo de referência bibliográfica em 'SILVA, João. A importância da educação online: desafios e oportunidades. Disponível em: https://www.educacaoonline.com.br/importancia-educacao-online/. Acesso em: 10 de março de 2023.'"
# consulta = "O que há de errado na referência bibliográfica 'João, SILVA. A importância da educação online: desafios e oportunidades. Disponível em: https://www.educacaoonline.com.br/importancia-educacao-online/. Acesso em: 10 de março de 2023.'"
prompt_create(consulta)

**Referência Bibliográfica**

**SOBRENOME, Nome. Título: subtítulo (se houver). Edição. Local de publicação: Editora, ano de publicação.**

**Exemplo:**

**SILVA, João. Metodologia da Pesquisa Científica: um guia prático. 5ª ed. São Paulo: Atlas, 2023.**

**Explicação:**

* **SOBRENOME, Nome:** Nome do autor(a) do livro.
* **Título:** Título principal do livro.
* **Subtítulo (se houver):** Título secundário que fornece informações adicionais sobre o conteúdo do livro.
* **Edição:** Número da edição do livro (se houver mais de uma).
* **Local de publicação:** Cidade onde o livro foi publicado.
* **Editora:** Nome da editora que publicou o livro.
* **Ano de publicação:** Ano em que o livro foi publicado.

**Tipos de Referências Bibliográficas:**

* **Livro:** O exemplo acima é uma referência bibliográfica para um livro.
* **Artigo de periódico:** SOBRENOME, Nome. Título do artigo. Nome do periódico, volume(número), páginas, ano de publicação.
* **Artigo de jornal:** SOBRENOME, Nome. Títu